<h2>
    <p>
        MÔN BIG DATA 2019
    </p>
    <p style="color:red">
        Nhóm 13 - Le Nguyen Son Nguyen - CH1702039
    </p>
</h2>

<h3>
    <p>
        SỬ DỤNG SPARK VÀ SPARK MACHINE LEARNING LIBRARY ĐỂ TRAIN MODEL PHÂN TÍCH NGỮ NGHĨA DÙNG MULTILAYER PERCEPTRONS (BASIC DEEP LEARNING)
    </p>
    <p style="color: blue">
        Phần đánh giá (Evaluation) Model đã train
    </p>
</h3>

<h4>
    <p>
        Khai báo các biến môi trường để Spark sử dụng trong quá trình chạy
    </p>
</h4>

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/spark/spark-2.4.3-bin-hadoop2.7"

<h4>
    <p style="color: blue">
        Dùng gói findSpark để khởi tạo môi trường chạy
    </p>
</h4>

In [2]:
import findspark
findspark.init()

<h4>
    <p>
        Import các thư viện của PySpark
    </p>
</h4>

In [3]:
from os import listdir
from pyspark import SparkContext
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import *

<h4>
    <p>
        Khai báo số memory mà mỗi Worker sẽ sử dụng để thực thi task
    </p>
</h4>

In [4]:
SparkContext.setSystemProperty('spark.executor.memory', '6656m')

<h4>
    <p>
        Khai báo địa chỉ của Spark master, lưu ý là phải thay đổi địa chỉ cho phù hợp với từng Spark cluster
    </p>
</h4>

In [5]:
# tạo spark context, lưu ý nhớ thay đổi lại địa chỉ của spark cho phù hợp
sc = SparkContext("spark://10.255.255.6:7077", "Sematic Analysic Evaluation")
spark = SparkSession.builder.getOrCreate()

In [6]:
# khai báo một số hằng số
NUMBER_INSTANCE_FOR_PC = 12500
# NUMBER_FEATURE_VECTOR = 500
# đường dẫn đến tập train positive
TEST_PATH_POS = "./semantic_data/aclImdb/test/pos/"
# đường dẫn đến tập train negative
TEST_PATH_NEG = "./semantic_data/aclImdb/test/neg/"
# biến lưu dữ liệu train cả positive và negative
data_test = []

<h4>
    <p style="color:blue">
        Dữ liệu test cũng có 2 tập:
    </p>
    <p style="color:red">
        Positive: nhận xét tích cực, chứa các câu nhận xét mang ý nghĩa tích cực, sẽ được đánh label là 1
    </p>
    <p style="color:red">
        Negative: nhận xét tiêu cực, chứa các câu nhận xét mang ý nghĩa tiêu cực, sẽ được đánh label là 0
    </p>
</h4>

In [7]:
# đọc dữ liệu của data train positive
listfile_pos = listdir(TEST_PATH_POS)
listfile_pos = enumerate(listfile_pos)
counter = 1
for index, file_pos in listfile_pos:
    try:
        with open(TEST_PATH_POS + file_pos, 'r') as f_pos:
            noidung = f_pos.read()
            noidung = noidung.lower()
            # đánh label cho tập data train positive là 1
            data_test.append((index, noidung, 1))
        if counter >= NUMBER_INSTANCE_FOR_PC:
            break
        else:
            counter = counter + 1
    except Exception as e:
        print(e)

In [8]:
# lấy dữ liệu cho tập data train negative
listfile_neg = listdir(TEST_PATH_NEG)
listfile_neg = enumerate(listfile_neg)
counter = 1
for index, file_neg in listfile_neg:
    try:
        with open(TEST_PATH_NEG + file_neg) as f_neg:
            noidung_neg = f_neg.read()
            noidung_neg = noidung_neg.lower()
            # đánh label cho data train negative là 0
            data_test.append((index, noidung_neg, 0))
            if counter >= NUMBER_INSTANCE_FOR_PC:
                break
            else:
                counter = counter + 1
    except Exception as e:
        print(e)

<h4>
    <p style="color:red">
        Tạo đối tượng DataFrame trong Spark để chứa dữ liệu train, đây là kiểu dữ liệu chuẩn trong việc xử lý phân tán và có thể thao tác dưới dạng SQL, lúc này dữ liệu train đã ở dạng phân tán khắp cluster
    </p>
</h4>

In [9]:
# tạo DataFrame trong Spark
sentenceDataFrame_test = spark.createDataFrame(data_test, ["id", "sentence", "label"])

<h4>
    <p>
      Sử dụng lại model đã lưu thì dùng lệnh dưới đây
    </p>
</h4>

In [10]:
sentenceDataFrame_test

DataFrame[id: bigint, sentence: string, label: bigint]

In [11]:
sentenceDataFrame_test.show(50)

+---+--------------------+-----+
| id|            sentence|label|
+---+--------------------+-----+
|  0|nothing really un...|    1|
|  1|i saw this movie ...|    1|
|  2|sharpe's honour f...|    1|
|  3|words can hardly ...|    1|
|  4|this small john f...|    1|
|  5|this film screene...|    1|
|  6|ghillie a remake ...|    1|
|  7|this is the best ...|    1|
|  8|i really enjoyed ...|    1|
|  9|the last film tha...|    1|
| 10|featuring some am...|    1|
| 11|this isn't a life...|    1|
| 12|i thought this mo...|    1|
| 13|this movie stands...|    1|
| 14|i was impressed w...|    1|
| 15|i consider myself...|    1|
| 16|rita hayworth as ...|    1|
| 17|i had the good fo...|    1|
| 18|i loved this film...|    1|
| 19|people who say th...|    1|
| 20|this may be the m...|    1|
| 21|i never saw docto...|    1|
| 22|hilarious show wi...|    1|
| 23|in following the ...|    1|
| 24|good movie, actin...|    1|
| 25|what makes this o...|    1|
| 26|writer/director j...|    1|
| 27|kon i

In [12]:
pl_model = PipelineModel.load("hdfs://s14e18f4e58324b66b78ecd8c831a0c6413-master.uitlab.com:9000/sparkdata/pl2.model")

<h4>
    <p style="color:red">
      Dùng model đã train áp dụng (transform) lên dữ liệu test đã tạo
    </p>
</h4>

In [13]:
data_after_transform = pl_model.transform(sentenceDataFrame_test)

In [14]:
data_after_transform.show(50)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|            sentence|label|           words_raw|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0|nothing really un...|    1|[nothing, really,...|[nothing, really,...|(500,[3,14,15,18,...|(500,[3,14,15,18,...|[-1.1581063945789...|[0.20211904987504...|       1.0|
|  1|i saw this movie ...|    1|[i, saw, this, mo...|[saw, movie, last...|(500,[3,5,6,40,45...|(500,[3,5,6,40,45...|[0.71244190024558...|[0.90394193650352...|       0.0|
|  2|sharpe's honour f...|    1|[sharpe's, honour...|[sharpe's, honour...|(500,[1,3,4,6,7,8...|(500,[1,3,4,6,7,8...|[0.91740188965713...|[0.9357141023

<h4>
    <p style="color:red">
      Tạo đối tượng Evaluator có sẵn trong Spark MLLib, đánh giá model
    </p>
</h4>

In [15]:
evaluator = MulticlassClassificationEvaluator()
result = evaluator.evaluate(data_after_transform)

<h4>
    <p>
      In ra kết quả đánh giá là độ chính xác, có thể áp dụng nhiều thông số đánh giá khác. Tham khảo thêm tại:
    </p>
    <p>
        <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier">https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier</a>
    </p>
</h4>

In [16]:
result

0.6917951668550886

<h4>
    <p>
        Kết Thúc một phiên làm việc trong Spark
    </p>
</h4>

In [17]:
print("end_of_code")
spark.stop()

end_of_code
